In [4]:
from fastapi import FastAPI, UploadFile, Form
from fastapi.middleware.cors import CORSMiddleware
from src.ocr import extract_titles
from src.metadata import fetch_metadata
from src.embeddings import generate_embedding
from src.recommender import recommend_books
from src.utils import save_file
import os
import easyocr

In [ ]:
image_path = "C:/Users/Samar/OneDrive/Documents/Coding practice/ShelfScanner/backend/data/shelf_images/a2b1928c-1df5-489d-a4ac-64fcbca1ca83_1000026200.jpg"
reader = easyocr.Reader(['en'])
result = reader.readtext(image_path, detail=0)
print(result)
# Simple heuristic: join all text as potential titles, split by lines or common separators
# titles = ' '.join(result).split('\n')  # Basic splitting; improve with vision if needed
# print([title.strip() for title in titles if title.strip()])

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
C:\Users\Samar\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


[':', '3', '2', 'I', 'AR', '8', '0', '6', '1', '3', '8', '1', '3', 'N A t ! 0 N $', 'NL', '<', 'Qu', '8', '4', '1', '1', 'A DA M', '0', 'M', '08', '0', 'S MI T H', 'OHapaTi', 'BHARAN', 'introduction By ALAN', 'B .', 'KROEGER', 'IBHAWAN', '(P', 'LD', 'eal', 'pnay eove', 'ELIZABETH GILBERT', 'LEGAL EAGLES', 'BHAX', 'H', 'Paulo Coelho', 'TlPortoblloof', '4', '2', '4', 'GIVE TAKE', '28', '1', '@2g8', '3', 'JACI HIGGINS', 'BAD GONIPAMY', 'J]', '0 &', '%', '83', '60', '40', 'The', 'Psychologyof Money', 'MORGAN HOUSEL', '88', 'THOMAS HARDY', 'umpty', 'Publications International Ltd', 'Selected Works', 'THE', 'Biography', 'TINY TOT', 'VKRAM VEIAVALL', 'RAM JETHMALANI', 'h', '2E', 'Our Environment', '48 Book Paradise(', 'REBEL', 'SUSAN ADELMAN', 'RAAAARGHI', 'JEREMY_STRONG', 'DINOSAUR', 'Pox', 'NANI PALKHIVALA', 'Soli J Sorabjee', 'BANDAGESI', 'JEREMY STRONG LET $', 'Do', 'THE', 'Pharaohi', 'GENIUS', 'Arvind P Datar', 'LexisNexis"', '6', 'COURTROOM', '0', 'NEW', 'dhisndzoDtsmixi', '9', 'DotoL',

In [8]:
titles = ' '.join(result).split('\n')
# print(titles)
print([title.strip() for title in titles if title.strip()])

[': 3 2 I AR 8 0 6 1 3 8 1 3 N A t ! 0 N $ NL < Qu 8 4 1 1 A DA M 0 M 08 0 S MI T H OHapaTi BHARAN introduction By ALAN B . KROEGER IBHAWAN (P LD eal pnay eove ELIZABETH GILBERT LEGAL EAGLES BHAX H Paulo Coelho TlPortoblloof 4 2 4 GIVE TAKE 28 1 @2g8 3 JACI HIGGINS BAD GONIPAMY J] 0 & % 83 60 40 The Psychologyof Money MORGAN HOUSEL 88 THOMAS HARDY umpty Publications International Ltd Selected Works THE Biography TINY TOT VKRAM VEIAVALL RAM JETHMALANI h 2E Our Environment 48 Book Paradise( REBEL SUSAN ADELMAN RAAAARGHI JEREMY_STRONG DINOSAUR Pox NANI PALKHIVALA Soli J Sorabjee BANDAGESI JEREMY STRONG LET $ Do THE Pharaohi GENIUS Arvind P Datar LexisNexis" 6 COURTROOM 0 NEW dhisndzoDtsmixi 9 DotoL IO RLd WIDE THP ONESTSEUEES WO RLD WHDE 1 2o 9 ROBERT 0 5 46 3 @2 3 V BESTSELLER JoHN DESTSELLER 1 1 LUDIM 1 g3 1 1 5 46 BLHILy GrUgham  BlMEY E VHHH 2 I0 3 31 3 4 3 0 6 2 1 CLEAREPRESEM CHAHEber TESUMF 1 IH APOCHPSE E1 3 HommanLeuis 8 H 1 DA_BER THE FEARG 1 1 IH L WATCH 1 3 I3/ 23 RAINMaker IV

In [12]:
import cv2
import numpy as np

In [ ]:
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [13]:
alpha = 1.5  # Contrast control (1.0-3.0)
beta = 50    # Brightness control (0-100)
contrasted = cv2.convertScaleAbs(gray, alpha=alpha, beta=beta)

In [14]:
_, binary = cv2.threshold(contrasted, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

In [15]:
denoised = cv2.GaussianBlur(binary, (5, 5), 0)

In [16]:
scale_percent = 150  # Increase by 50%
width = int(denoised.shape[1] * scale_percent / 100)
height = int(denoised.shape[0] * scale_percent / 100)
resized = cv2.resize(denoised, (width, height), interpolation=cv2.INTER_CUBIC)

In [18]:
result = reader.readtext(resized, detail=0)
# print(result)
# Simple heuristic: join all text as potential titles, split by lines or common separators
titles = ' '.join(result).split('\n')  # Basic splitting; improve with vision if needed
print([title.strip() for title in titles if title.strip()])

['# 7 0 4 A 2 f 1 AS S M ] T % 00 itroducfiow %" Gii7 2 pcy loxe ELIzABETh GILBERT: 1 ` L- BHAN Paulo Coelho Tortobuloet 33 YE64d z 0 JACU ES 3 0 _ ~0 2 Du 107\' Mukuanhuusel 33aS HARDY THE IOT VOYIWZ MALANI 8 F REB DELMAN WEnOSAUR Pow NAN Sol Sora LANdAGES\' ZEREY NG LEt*$ DO THE phara@hl COURTROOM GENIUS Arvind P Datar LexisNexis" 6 NEW bindzodi_au 6 @ @ ui WOBLOMIde 1 ROBERT 0 D 0 3 12 Mau 1 1 2 LudLuM 8 F B VLUUDA MMu U 1 2 401 1 9 2 ~ 1 ELvB i; CHAHEBER Tm a1 Kzeh APOCHESE 1 7 Norman Lcrzs 0 DHncR RanhEker #4 [ WATCA 1 1 33 JF De GOL SaB 0 3 ECTCE TZETE 413 Turnip 1 1 0 SHl 9 3 1 1 0 1']
